Computing for bond length: 3.7241379310344827 Å
Total ground state energy = [-55.82411139]
condition number: 124.43166749451767

Computing for bond length: 2.526315789473684 Å
Total ground state energy = [-55.82194694]
condition number: 427.6272134384225

In [1]:
import numpy as np
from qiskit import transpile
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA,SLSQP, POWELL
from qiskit.primitives import Estimator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector, SparsePauliOp
import scipy
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.circuit import ParameterVector
import numpy as np
from pyscf import gto, scf
import pyscf.mcscf as mcscf
import warnings
warnings.filterwarnings("ignore")


In [2]:
#Create the all possible excitations
num_spartial_orbital = 4
num_spin_orbitals = num_spartial_orbital * 2

# Initialize the mapper
mapper = JordanWignerMapper()
#Create an identity operator
I = FermionicOp({'': 1.0}, num_spin_orbitals =num_spin_orbitals)
I = mapper.map(I)

 #list of occupied orbitals
occupied = []
for i in range(num_spin_orbitals//4):
    occupied.append(i)
    occupied.append(i+num_spin_orbitals//2)
#occupied = [0,4,1,5]
    
# Generate all possible single excitations
excitations = []
def all_excitations(num_spin_orbitals):
    for i in range(num_spin_orbitals):
        for j in range(i+1, num_spin_orbitals):
            # Prevent electrons from moving from alpha spin to beta spin and beta spin to alpha
            if i != j and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2)):
                # Only consider excitations where the first two alpha and beta spins are filled with electrons
                if (i in occupied and j not in occupied): 
                    excitation = FermionicOp({f'+_{j} -_{i}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                    excitations.append(excitation)
    
        #Generate possible double excitations
        #Double excitations all from alpha or beta orbitals
            for k in range(j+1, num_spin_orbitals):
                for l in range(k+1, num_spin_orbitals):
                    if i != j and k != l and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2 and k < num_spin_orbitals // 2 and l < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2 and k >= num_spin_orbitals // 2 and l >= num_spin_orbitals // 2)):
                        # Only consider excitations where the first two alpha and beta spins are filled with electrons
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                            excitation = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(excitation)
  
    for i in range(num_spin_orbitals // 2):
        for j in range(num_spin_orbitals // 2, num_spin_orbitals):
            for k in range(num_spin_orbitals // 2):
                for l in range(num_spin_orbitals // 2, num_spin_orbitals):
                    if i != k and j != l and i < k and j < l:
                        # Condition to ensure one alpha and one beta excitation
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                           # Create the FermionicOp and add to double_exc list
                            exc = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(exc)

    return excitations
excitations = all_excitations(num_spin_orbitals)

In [3]:
eigval = []
#distances = np.linspace(0.60, 2.46, 19)
distances = np.linspace(0.4, 2.5, 30)
for d in distances:
    #atom = f"N 0.0 0.0 0.0; H {d} 0.0 0.0; H -0.506, 0.876 0.0; H -0.506 -0.876 0.0"
    atom = f"N 0.0 0.0 0.0; N {d} 0.0 0.0"
    basis = "sto-6g"
    driver = PySCFDriver(atom=atom,  basis=basis)

    problem = driver.run()

    active_electrons = 4
    active_transformer = ActiveSpaceTransformer(num_electrons=active_electrons, num_spatial_orbitals=num_spartial_orbital)
    active_problem = active_transformer.transform(problem)

    seed = 170
    algorithm_globals.random_seed = seed
    
    mol = gto.M(atom = atom, basis=basis, verbose = 0)
    mf = scf.RHF(mol).run()
    mc = mcscf.CASCI(mf, ncas=num_spartial_orbital, nelecas=active_electrons)
    mc.kernel()

    frozen_core_energy = mc.e_tot - mc.e_cas
    core = frozen_core_energy
    #print("Frozen-core energy =", core)


    # Initialize the mapper
    mapper = JordanWignerMapper()
            
    # Map the electronic problem to a qubit operator
    qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
            
    # Initialize the UCCSD ansatz with Hartree-Fock initial state
    ansatz = UCCSD(
        active_problem.num_spatial_orbitals,
        active_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            active_problem.num_spatial_orbitals,
            active_problem.num_particles,
            mapper
        ),
    )

    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
 
    
    solver = GroundStateEigensolver(mapper, vqe)
    result = solver.solve(active_problem)
    print(f"Computing for bond length: {d:.2f} Å")

    gr = result.total_energies

    print(f"ground state energy = {gr}")
    # Extract the ground state wavefunction parameters

    psi_vqe = result.raw_result.optimal_point
    ansatz.assign_parameters(psi_vqe, inplace=True) 

    from qiskit_aer.primitives import Estimator
    #for shots in [10**i for i in range(3, 4)]:
    estimator = Estimator(
                run_options={"shots": 1000},
                transpile_options={"seed_transpiler": 42},
            approximation=True)

    eigenvalues_1k = []
    # Initialize the matrix M
    num_excitations = len(excitations)
    M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    # Compute the matrix elements
    #for t in range(10):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
                        
    eig, ev = scipy.linalg.eigh(M, S) 
    eigval.append(eig + core)
    print("eigenvalues1k", eigval)

Computing for bond length: 0.40 Å
ground state energy = [-95.62084232]
eigenvalues1k [array([-95.61928538, -95.08302085, -95.08138164, -95.02462462,
       -95.02308509, -94.67506148, -94.66812718, -94.50100079,
       -94.47332963, -94.466525  , -94.44169494, -94.44135101,
       -94.39035455, -94.33369304, -94.14422028, -94.14390999,
       -94.11480106, -94.08460392, -93.94428731, -93.82035315,
       -93.8013893 , -93.80109722, -93.74179171, -93.51296734,
       -93.45496335, -93.45364889, -93.38914821])]
Computing for bond length: 0.47 Å
ground state energy = [-98.58777921]


LinAlgError: The leading minor of order 15 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.